#### Intro:
This project is for DiDi's competition. The objective is to predict the gap at certain times of the day, guranteed that the order information for the past 30 minutes is available.

First we want to have benchmarks or methods to evaluate the forcasting model. There are four method that we will test our models against. These are:

1- The Average Method.<br />
    Function: avg(a) <br />
    Residual: 403319.11

2- The Naive Method.<br />
    Function: naive(a,t) <br />
    <b>Residual: 153494</b>

3- The Seasonal Naive Method.<br />
    Function: season_naive(a) <br />
    Residual: 163499

4- The Drift Method.<br />
    Function: drift(a)

In [2]:
# Import pylab to provide scientific Python libraries (NumPy, SciPy, Matplotlib)
%pylab --no-import-all
#import pylab as pl
# import the Image display module
from IPython.display import Image
import itertools
# inline allows us to embed matplotlib figures directly into the IPython notebook
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from datetime import datetime
from os import listdir
from os.path import isfile, join
from numpy.linalg import norm

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


/home/sahabi/miniconda/envs/data/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
def avg(a):
    return np.mean(a, axis=0)

def naive(a,t):
    return a[a.shape[0]-t,:]

def convert_date(t):
    d = datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    day = int(d.strftime('%d'))
    hour = int(d.strftime('%H'))
    minute = int(d.strftime('%M'))
    index = (day-1)*144 + ((minute%10)+1) + hour*6
    return index

In [ ]:
frames = []
mypath = '/media/sahabi/Shared Storage/season_1/training_data/order_data'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for i,files in enumerate(onlyfiles):
    data = '/media/sahabi/Shared Storage/season_1/training_data/order_data/'+files
    order_data = pd.read_csv(data,sep='\t',names=['order_id','driver_id','passenger_id','start_district_hash','dest_district_hash','Price','Time'])
    frames.append(order_data)
    if i ==140:
        break

result = pd.concat(frames)
order_data=result.fillna(0)
order_data['t_ind'] = order_data.Time.apply(convert_date)

In [ ]:
districts = order_data['start_district_hash'].unique()
districts = ['4725c39a5e5f4c188d382da3910b3f3f']
for j in districts:
    demand = []
    supply = []
    gap = []
    for i in order_data['t_ind'].unique():
        try:
            demand.append(order_data['start_district_hash'][(order_data.t_ind == i) & (order_data.start_district_hash==j)].value_counts()[0])
        except:
            demand.append(0)
        try:
            supply.append(order_data['start_district_hash'][(order_data.t_ind == i) & (order_data.start_district_hash==j) & 
                                                        (order_data.driver_id!=0)].value_counts()[0])
        except:
            supply.append(0)
        try:
            gap.append(order_data['start_district_hash'][(order_data.t_ind == i) & (order_data.start_district_hash==j) & 
                                                      (order_data.driver_id==0)].value_counts()[0])
        except:
            gap.append(0)

In [ ]:
a = np.array([demand[:len(demand)-3],supply[:len(supply)-3],gap[:len(gap)-3],demand[1:len(demand)-2],
             supply[1:len(supply)-2],gap[1:len(gap)-2],demand[2:len(demand)-1],supply[2:len(supply)-1],
             gap[2:len(gap)-1]])

y_avg = avg(a)
y_naive = naive(a,1)
ts_avg = np.concatenate((gap[:3], y_avg))
ts_naive = np.concatenate((gap[:3],y_naive))
ts_season_naive = np.concatenate((gap[:3],y_naive[:1005],gap[:1008],gap[1008:2016]))
figure(3,figsize=(8, 6))
plot(gap - ts_naive)
plt.title('Naive ResidualR')
figure(4,figsize=(8, 6))
plot(gap-ts_avg)
plt.title('Average R')
figure(5,figsize=(8, 6))
plot(gap[0:3024]-ts_season_naive)
plt.title('Seasonal Naive R')

print 'Average: {}'.format(norm(ts_avg[0:3024] - gap[0:3024],1))
print 'Naive: {}'.format(norm(ts_naive[0:3024] - gap[0:3024],1))
print 'Seasonal Naive: {}'.format(norm(ts_season_naive - gap[0:3024],1))